# DASHBOARD
__This notebook is meant setup a training session and evaluate its performances__

----

Load python libraries

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
import keras.losses as klosses
import keras.optimizers as koptimizers
import keras.callbacks as kcallbacks
import keras.initializers as initializers

Using TensorFlow backend.


Load and setup local dependencies

In [2]:
cur_dir = os.getcwd()
base_dir = os.path.dirname(cur_dir)

import sys
sys.path.append(base_dir)
from utils.LungsLoader import LungsLoader
from utils.ScanHandler import ScanHandler

from src.training.config_file import ConfigFile
from src.evaluation.luna_testing import LunaTester

import warnings
warnings.filterwarnings("ignore")

loader = LungsLoader()
handler = ScanHandler(plt)

/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 0 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 1 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 2 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 3 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 4 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documen

# Initialize a session

In [5]:
session_name = "sandbox_session"
config = ConfigFile(session_name)
config.setup_session()

# Build the model

In [3]:
from src.networks.networks_utils import blocks
from src.networks.MariaNet import MariaNet

__Setting up model hyperparameters :__

In [6]:
input_shape = (64, 64, 64)

conv_block_kwgs = {
                    "activation": "LeakyReLU",
                    "activation_kwargs":{
                        "alpha": 0.3
                        },
                    "normalize": True,
                    "conv_kwargs": {
                      "kernel_size": 3,
                      "padding": "same",
                      "kernel_initializer": initializers.RandomNormal(mean=0.0, stddev=1e-5)
                      }
                    }
squeeze_ratio = 16
conv_block = blocks.ConvBlock(**conv_block_kwgs)
squeeze_block = blocks.SqueezeExciteBlock(ratio=squeeze_ratio)

enc_filters = [32, 64, 128, 32, 32]
enc_dilation = [(1, 1, 1), (1, 1, 1), (2, 2, 2), (3, 3, 3), (5, 5, 5)]
enc_params = [{"filters": n_filter, "dilation_rate": dil_rate} for (n_filter, dil_rate) in zip(enc_filters, enc_dilation)]

dec_filters = [128, 64, 32, 32, 32]
dec_params = [{"filters": n_filter} for n_filter in dec_filters]

def_flow_nf = 3
lin_flow_nf = 12

marianet = MariaNet(input_shape,
                    enc_params,
                     dec_params,
                     conv_block,
                     squeeze_block,
                     def_flow_nf,
                     lin_flow_nf)

__Serialize model builder__

In [7]:
builder_path = os.path.join(config.session_dir, "builder.pickle")
marianet.serialize(builder_path)

# Traning parameters

__Input Shape :__

In [8]:
config.set_input_shape(input_shape)

__Losses :__

In [9]:
def registration_loss(vol1, vol2):
    return klosses.mean_squared_error(vol1, vol2) - 1. * metrics.cross_correlation(vol1, vol2)


losses = [registration_loss, klosses.mean_absolute_error, klosses.mean_absolute_error]
loss_weights = [1., 1e-5, 1e-5]

config.set_losses(losses)
config.set_loss_weights(loss_weights)

__Optimizer :__

In [10]:
optimizer = koptimizers.Adam(lr=1e-2, decay=1e-5)
config.set_optimizer(optimizer)

__Callbacks :__

In [11]:
checkpoints_dir = os.path.join(config.session_dir, ConfigFile.checkpoints_dirname)
tensorboard_dir = os.path.join(config.session_dir, ConfigFile.tensorboard_dirname)


save_callback = kcallbacks.ModelCheckpoint(os.path.join(checkpoints_dir, ConfigFile.checkpoints_format), 
                                           verbose=1, 
                                           save_best_only=True)
early_stopping = kcallbacks.EarlyStopping(monitor='val_loss',
                                          min_delta=1e-3,
                                          patience=20,
                                          mode='auto')
tbCallBack = kcallbacks.TensorBoard(log_dir=tensorboard_dir, 
                                    histogram_freq=0, 
                                    write_graph=True, 
                                    write_images=True)

callbacks = [save_callback, early_stopping, tbCallBack]
config.set_callbacks(callbacks)

__Training scope :__

In [12]:
epochs = 100
steps_per_epoch = 50

config.set_epochs(epochs)
config.set_steps_per_epoch(steps_per_epoch)

__Serialize session configs__

In [13]:
config.serialize()